# Neural Network's Project: Causal Explanations of Image Misclassifications

# Import Libraries

In [ ]:
# %tensorflow_version 1.x
!pip install keras==2.3.1
!pip install keras-metrics

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import sys
import random
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras_metrics as km
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.models import Model, Input
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import seaborn as sns
from PIL import Image
import sklearn.metrics 
from sklearn import svm
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.multiclass import unique_labels
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, GridSearchCV
from google.colab import drive
from google.colab import files


print('Tensorflow ', tf.__version__)
print('Keras ', keras.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

print("Libraries imported")

# Load Dataset

In [ ]:
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt



from google.colab import files 

(x_train,y_train),(x_test,y_test) = cifar10.load_data()


SPLIT

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:

print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

In [ ]:
K = len(set(y_train))
print("number of classes:", K)

# CNN Models

In [ ]:
input_shape = (x_train[0].shape)
n_classes = 10 


NICKNET


In [ ]:
def Nicknet(input_shape,n_classes):
    regl2 = 0.0001
    lr = 0.0001
    model = Sequential()

   
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(15,15),\
                     strides=(2,4), padding='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=256, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
  
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=384, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
   
    model.add(BatchNormalization())

    
    model.add(Conv2D(filters=384, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=256, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(BatchNormalization())

    
    model.add(Flatten())

    flatten_shape = (input_shape[0]*input_shape[1]*input_shape[2],)
    
    
    model.add(Dense(4096, input_shape=flatten_shape, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(1000,kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    

    adam = optimizers.Adam(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy',km.precision(),km.recall()])

    return model

model = Nicknet(input_shape,n_classes) 
model.summary()


NickNet 2

In [ ]:
def Nicknet2(input_shape,n_classes):
    regl2 = 0.0001
    lr = 0.0001
    model = Sequential()

   
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(15,15),\
                     strides=(2,4), padding='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=256, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
  
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=384, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
   
    model.add(BatchNormalization())

    
    model.add(Conv2D(filters=384, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=256, kernel_size=(15,15), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(BatchNormalization())

    
    model.add(Flatten())

    flatten_shape = (input_shape[0]*input_shape[1]*input_shape[2],)
    
    
    model.add(Dense(4096, input_shape=flatten_shape, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(1000,kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    

    sgd = optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy',km.precision(),km.recall()])

    return model

model = Nicknet2(input_shape,n_classes) 
model.summary()


NickNet3

In [ ]:
def Nicknet3(input_shape,n_classes):
    regl2 = 0.0001
    lr = 0.0001
    model = Sequential()

   
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(10,10),\
                     strides=(2,4), padding='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=256, kernel_size=(10,10), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
  
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=384, kernel_size=(10,10), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
   
    model.add(BatchNormalization())

    
    model.add(Conv2D(filters=384, kernel_size=(10,10), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    
    model.add(BatchNormalization())

   
    model.add(Conv2D(filters=256, kernel_size=(10,10), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(BatchNormalization())

    
    model.add(Flatten())

    flatten_shape = (input_shape[0]*input_shape[1]*input_shape[2],)
    
    
    model.add(Dense(4096, input_shape=flatten_shape, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(1000,kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))
    
    model.add(BatchNormalization())

    
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    

    sgd = optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy',km.precision(),km.recall()])

    return model

model = Nicknet3(input_shape,n_classes) 
model.summary()


# Training 


In [ ]:
batch_size = 128
epochs = 30


history = model.fit(x_train,y_train, batch_size = batch_size , epochs=epochs,
                    verbose=1,
                    validation_data=(x_test,y_test),
          )

# METRICS


CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
plt.rcParams['figure.figsize'] = [10,7]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()


p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('model precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# DEFINE THE LABELS


In [ ]:
labels = '''airplane
automobile
bird
cat
deer
dog
frog
horse
ship
truck'''.split()

Check The right predictions

In [ ]:
misclassified_idx = np.where(p_test == y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s Predicted: %s" % (labels[y_test[i]], labels[p_test[i]]));

Check the wrong predictions

In [ ]:
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s Predicted: %s" % (labels[y_test[i]], labels[p_test[i]]));